# Challenge1 
## Group 5
## 202318007, 202318043, 202318052, 202318062

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
from sklearn.neighbors import NearestNeighbors


In [4]:
df = pd.read_csv('D:/Recommender System Datasets/challenge1/rating.csv')
df

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [5]:
df1 = df[['userId', 'movieId', 'rating']]

In [6]:
df1 = df[['userId', 'movieId', 'rating']]
df1_rating_tm = pd.DataFrame(df1.groupby('rating').size(), columns = ['count'])
df1_rating_tm

,count
rating,
0.5,239125
1.0,680732
1.5,279252
2.0,1430997
2.5,883398
3.0,4291193
3.5,2200156
4.0,5561926
4.5,1534824


In [7]:
def shrinking_data(no_of_top_users):
    user_likes = df1.groupby('userId')['movieId'].count()
    top_users = user_likes.nlargest(no_of_top_users).index
    print(f"Top Users Index Values: {top_users}")
    top_users_df = df1[df1['userId'].isin(top_users)].reset_index(drop=True)
    print(f'Top {no_of_top_users} Dataframe')
    return top_users_df

In [8]:
no_of_top_users = 10000
final_top_users_df = shrinking_data(no_of_top_users)
final_top_users_df 

Top Users Index Values: Int64Index([118205,   8405,  82418, 121535, 125794,  74142,  34576, 131904,
             83090,  59477,
            ...
             92433,  95536, 102921, 103578, 106776, 108493, 110559, 115351,
            116360, 117537],
           dtype='int64', name='userId', length=10000)
Top 10000 Dataframe


,userId,movieId,rating
0,11,1,4.5
1,11,10,2.5
2,11,19,3.5
3,11,32,5.0
4,11,39,4.5
...,...,...,...
7718626,138474,5401,1.0
7718627,138474,5449,4.0
7718628,138474,5459,4.0
7718629,138474,5460,5.0


In [9]:
features_train, features_test, target_train, target_test = train_test_split(final_top_users_df[['userId', 'movieId']],final_top_users_df['rating'], test_size=0.2, random_state=42, stratify = final_top_users_df['rating'])
df_shrink_trainset = pd.concat([features_train, target_train], axis = 1).reset_index(drop=True)
df_shrink_testset = pd.concat([features_test, target_test], axis = 1).reset_index(drop=True)

In [10]:
def create_mappings(dataframe_name):
 
    user2movie = dataframe_name.groupby('userId')['movieId'].unique().to_dict()

    movie2user = dataframe_name.groupby('movieId')['userId'].unique().to_dict()

    return user2movie, movie2user

user2movie, movie2user = create_mappings(df_shrink_trainset)

In [11]:
def usermovierating_mappings(dataframe_name):
    usermovie2rating = dataframe_name.pivot_table(index='userId', columns='movieId', values='rating')
    
    usermovie2rating.fillna(0, inplace=True)
    
    
    return usermovie2rating

In [12]:
usermovie_to_rating_train = usermovierating_mappings(df_shrink_trainset)
usermovie_to_rating_train

Processing time: 26.998650074005127 seconds


movieId,1,2,3,4,5,6,7,8,9,10,...,131172,131176,131180,131231,131239,131243,131248,131258,131260,131262
userId,,,,,,,,,,,,,,,,,,,,,
11,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,4.0,0.0,0.0,0.0,2.0,4.0,3.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,4.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,5.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,0.0,3.5,3.0,0.0,0.0,0.0,2.5,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138404,5.0,2.5,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138406,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138411,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
usermovie_to_rating_test = usermovierating_mappings(df_shrink_testset)
usermovie_to_rating_test

Processing time: 11.863194465637207 seconds


movieId,1,2,3,4,5,6,7,8,9,10,...,131158,131164,131166,131168,131174,131241,131250,131252,131254,131256
userId,,,,,,,,,,,,,,,,,,,,,
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138406,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138411,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
cf_knn_model= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
cf_knn_model.fit(usermovie_to_rating_train)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [23]:
cf_knn_model.kneighbors(usermovie_to_rating_train.T.iloc[:, 5].values.reshape(1, -1))

(array([[0.        , 0.56790177, 0.56900183, 0.56950579, 0.5710634 ,
         0.57281101, 0.57320112, 0.57581834, 0.57687794, 0.58303161]]),
 array([[   5, 5348, 9095, 9328, 5359, 9610, 7564, 9961, 5820, 5667]],
       dtype=int64))

In [15]:
movie_metadata = pd.read_csv("D:/Recommender System Datasets/challenge1/movie.csv")
movie_metadata.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
def datamerge(df1, df2):
    merging = df1.merge(df2, on='movieId')
    return merging
movie_data = datamerge(df_shrink_trainset, movie_metadata)
movie_data.head(5)

,userId,movieId,rating,title,genres
0,11924,34405,2.0,Serenity (2005),Action|Adventure|Sci-Fi
1,118372,34405,4.5,Serenity (2005),Action|Adventure|Sci-Fi
2,38669,34405,4.0,Serenity (2005),Action|Adventure|Sci-Fi
3,20158,34405,4.5,Serenity (2005),Action|Adventure|Sci-Fi
4,100804,34405,3.5,Serenity (2005),Action|Adventure|Sci-Fi


In [17]:
movie = pd.read_csv("D:/Recommender System Datasets/challenge1/movie.csv")
movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [18]:
usermovie_to_rating_train.T

userId,11,24,54,58,91,96,104,116,131,134,...,138317,138325,138335,138382,138397,138404,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.5,4.0,4.0,5.0,0.0,3.5,0.0,3.0,2.0,4.0,...,3.0,5.0,0.0,3.0,0.0,5.0,4.0,0.0,4.0,5.0
2,0.0,0.0,3.0,0.0,3.5,0.0,0.0,2.0,1.0,0.0,...,3.0,3.0,0.0,4.0,0.0,2.5,0.0,0.0,0.0,4.0
3,0.0,0.0,0.0,0.0,3.0,4.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
def movie_recommendation(movie_id, no_of_nearest_neighbors):
    #no_of_nearest_neighbors=no_of_nearest_neighbors+1
    distances, indices = cf_knn_model.kneighbors(usermovie_to_rating_train.T.iloc[:, movie_id].values.reshape(1, -1))
    similar_movies_indices = indices.flatten()[1:]  
    similar_movies_ids = usermovie_to_rating_train.columns[similar_movies_indices].tolist()
    print(f'similar_movie_ids: {similar_movies_ids}')
    print(distances[0])
    cf_recs = []
    c=1
    for i in similar_movies_ids:
            cf_recs.append({'Movie Id': i, 'Title':movie['title'][i], 'Cosine_Similarity':distances[0][c]})
            c=c+1
    df = pd.DataFrame(cf_recs, index = range(1,no_of_nearest_neighbors))
    
    return  df

chosen_movie_id = 10
recommended_movies = movie_recommendation(chosen_movie_id, 10)  
print('chosen movie: ', movie['title'][chosen_movie_id])
print("Recommendations:") 
recommended_movies

similar_movie_ids: [6116, 3023, 3238, 8405, 6761, 7145, 1947, 8698, 69]
[6.66133815e-16 4.91647184e-01 5.16892711e-01 5.35320127e-01
 5.35883911e-01 5.37412520e-01 5.40651963e-01 5.45484834e-01
 5.46749387e-01 5.48178276e-01]
chosen movie:  American President, The (1995)
Recommendations:


,Movie Id,Title,Cosine_Similarity
1,6116,Laurel Canyon (2002),0.491647
2,3023,Country (1984),0.516893
3,3238,"Next Best Thing, The (2000)",0.535320
4,8405,"Raven, The (1935)",0.535884
5,6761,Good Boy! (2003),0.537413
6,7145,"Big Bounce, The (2004)",0.540652
7,1947,"Million Dollar Duck, The (a.k.a. $1,000,000 Du...",0.545485
8,8698,Gamera vs. Jiger (1970),0.546749
9,69,From Dusk Till Dawn (1996),0.548178


In [36]:
cf_knn_model= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5, n_jobs=-1)
cf_knn_model.fit(usermovie_to_rating_train)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [37]:
def movie_recommendation(movie_id, no_of_nearest_neighbors):
    #no_of_nearest_neighbors=no_of_nearest_neighbors+1
    distances, indices = cf_knn_model.kneighbors(usermovie_to_rating_train.T.iloc[:, movie_id].values.reshape(1, -1))
    similar_movies_indices = indices.flatten()[1:]  
    similar_movies_ids = usermovie_to_rating_train.columns[similar_movies_indices].tolist()
    print(f'similar_movie_ids: {similar_movies_ids}')
    print(distances[0])
    cf_recs = []
    c=1
    for i in similar_movies_ids:
            cf_recs.append({'Movie Id': i, 'Title':movie['title'][i], 'Cosine_Similarity':distances[0][c]})
            c=c+1
    df = pd.DataFrame(cf_recs, index = range(1,no_of_nearest_neighbors))
    
    return  df

chosen_movie_id = 10
recommended_movies = movie_recommendation(chosen_movie_id, 5)  
print('chosen movie: ', movie['title'][chosen_movie_id])
print("Recommendations:") 
recommended_movies

similar_movie_ids: [6116, 3023, 3238, 8405]
[6.66133815e-16 4.91647184e-01 5.16892711e-01 5.35320127e-01
 5.35883911e-01]
chosen movie:  American President, The (1995)
Recommendations:


,Movie Id,Title,Cosine_Similarity
1,6116,Laurel Canyon (2002),0.491647
2,3023,Country (1984),0.516893
3,3238,"Next Best Thing, The (2000)",0.535320
4,8405,"Raven, The (1935)",0.535884


In [38]:
recommended_movies['Cosine_Similarity'].sum()

2.079743933296318

In [43]:
# Cosine Similarities for choice of different number of Neighbors 

k=2
for k in range(10,20):
    cf_knn_model= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k, n_jobs=-1)
    cf_knn_model.fit(usermovie_to_rating_train)
    chosen_movie_id = 10
    recommended_movies = movie_recommendation(chosen_movie_id, k)  
    print("For k = ",k)
    print(recommended_movies['Cosine_Similarity'].sum()/(k-1))
    
    

similar_movie_ids: [6116, 3023, 3238, 8405, 6761, 7145, 1947, 8698, 69]
[6.66133815e-16 4.91647184e-01 5.16892711e-01 5.35320127e-01
 5.35883911e-01 5.37412520e-01 5.40651963e-01 5.45484834e-01
 5.46749387e-01 5.48178276e-01]
For k =  10
0.5331356571317071
similar_movie_ids: [6116, 3023, 3238, 8405, 6761, 7145, 1947, 8698, 69, 2271]
[6.66133815e-16 4.91647184e-01 5.16892711e-01 5.35320127e-01
 5.35883911e-01 5.37412520e-01 5.40651963e-01 5.45484834e-01
 5.46749387e-01 5.48178276e-01 5.52467048e-01]
For k =  11
0.5350687962104452
similar_movie_ids: [6116, 3023, 3238, 8405, 6761, 7145, 1947, 8698, 69, 2271, 5782]
[6.66133815e-16 4.91647184e-01 5.16892711e-01 5.35320127e-01
 5.35883911e-01 5.37412520e-01 5.40651963e-01 5.45484834e-01
 5.46749387e-01 5.48178276e-01 5.52467048e-01 5.53059328e-01]
For k =  12
0.5367042991230837
similar_movie_ids: [6116, 3023, 3238, 8405, 6761, 7145, 1947, 8698, 69, 2271, 5782, 2128]
[6.66133815e-16 4.91647184e-01 5.16892711e-01 5.35320127e-01
 5.35883911e-01